In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import heapq
import time
import psutil
from collections import defaultdict
import os
from numba import jit
import pandas as pd
import seaborn as sns
import tracemalloc

@jit(nopython=True)
def heuristic(a, b):
    return abs(b[0] - a[0]) + abs(b[1] - a[1])  # Manhattan distance

@jit(nopython=True)
def get_neighbors(grid, x, y):
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and grid[nx, ny] == 0:
            neighbors.append((nx, ny))
    return neighbors

@jit(nopython=True)
def create_search_area(grid, line_array, band_width):
    search_area = np.zeros_like(grid, dtype=np.bool_)
    for i in range(line_array.shape[0]):
        x, y = line_array[i]
        x_min, x_max = max(0, x-band_width), min(grid.shape[0], x+band_width+1)
        y_min, y_max = max(0, y-band_width), min(grid.shape[1], y+band_width+1)
        search_area[x_min:x_max, y_min:y_max] = True
    return search_area

@jit(nopython=True)
def astar_numba(grid, start, goal, search_area, max_iterations):
    rows, cols = grid.shape
    open_list = [(0, start[0], start[1], 0, -1, -1)]  # (f, x, y, g, parent_x, parent_y)
    closed_set = np.zeros((rows, cols), dtype=np.bool_)
    g_scores = np.full((rows, cols), np.inf)
    g_scores[start] = 0
    nodes_visited = 0

    while open_list and nodes_visited < max_iterations:
        nodes_visited += 1
        current = heapq.heappop(open_list)
        _, x, y, g, _, _ = current

        if (x, y) == goal:
            path = []
            while (x, y) != start:
                path.append((x, y))
                px, py = int(g_scores[x-1, y]), int(g_scores[x, y-1])
                if x > 0 and px < py:
                    x -= 1
                else:
                    y -= 1
            path.append(start)
            return path[::-1], nodes_visited

        closed_set[x, y] = True

        for nx, ny in get_neighbors(grid, x, y):
            if closed_set[nx, ny] or not search_area[nx, ny]:
                continue

            tentative_g = g + 1

            if tentative_g < g_scores[nx, ny]:
                g_scores[nx, ny] = tentative_g
                f = tentative_g + heuristic((nx, ny), goal)
                heapq.heappush(open_list, (f, nx, ny, tentative_g, x, y))

    return None, nodes_visited

def bresenham_line(start, end):
    x1, y1 = start
    x2, y2 = end
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    err = dx - dy
    
    line_points = []
    while True:
        line_points.append((x1, y1))
        if x1 == x2 and y1 == y2:
            break
        e2 = 2 * err
        if e2 > -dy:
            err -= dy
            x1 += sx
        if e2 < dx:
            err += dx
            y1 += sy
    return np.array(line_points, dtype=np.int32)

def optimized_incremental_astar(grid, start, goal, initial_band_width=4, expansion_factor=2, max_iterations=100000):
    line_array = bresenham_line(start, goal)
    band_width = initial_band_width
    nodes_visited_total = 0
    max_band_width = max(grid.shape)

    while band_width <= max_band_width:
        search_area = create_search_area(grid, line_array, band_width)
        path, nodes_visited = astar_numba(grid, start, goal, search_area, max_iterations)
        nodes_visited_total += nodes_visited

        if path:
            return path, nodes_visited_total

        if band_width == max_band_width:
            print("No path found after exploring the entire grid.")
            return None, nodes_visited_total

        band_width = min(int(band_width * expansion_factor), max_band_width)
        print(f"Path not found. Expanding search area. New band width: {band_width}")

    return None, nodes_visited_total

def image_to_grid(image_path, threshold=128):
    img = Image.open(image_path).convert('L')
    grid = np.array(img)
    grid = (grid < threshold).astype(np.int32)
    return grid

def draw_path(image_path, path):
    img = Image.open(image_path).convert('RGB')
    img_array = np.array(img)

    for x, y in path:
        img_array[x, y] = [255, 0, 0]  # Red color for the path

    return Image.fromarray(img_array)

def run_algorithm(algorithm, grid, start, goal, **kwargs):
    tracemalloc.start()
    start_time = time.time()
    
    path, nodes_visited = algorithm(grid, start, goal, **kwargs)
    
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    execution_time = end_time - start_time
    memory_usage = peak / 1024 / 1024  # Convert to MB

    return path, nodes_visited, execution_time, memory_usage

def process_image(image_path):
    grid = image_to_grid(image_path)
    start = (0, 0)
    goal = (grid.shape[0] - 1, grid.shape[1] - 1)

    print(f"\nProcessing image: {image_path}")
    print(f"Image size: {grid.shape[0]}x{grid.shape[1]}")
    print(f"Start point: {start}")
    print(f"Goal point: {goal}")

    astar_path, astar_nodes, astar_time, astar_memory = run_algorithm(
        astar_numba, grid, start, goal, 
        search_area=np.ones_like(grid, dtype=np.bool_), 
        max_iterations=100000
    )
    iba_path, iba_nodes, iba_time, iba_memory = run_algorithm(
        optimized_incremental_astar, grid, start, goal
    )

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    
    ax1.imshow(Image.open(image_path))
    ax1.set_title('Original Image')
    ax1.axis('off')

    if astar_path:
        ax2.imshow(draw_path(image_path, astar_path))
        ax2.set_title('A* Path')
        ax2.axis('off')

    if iba_path:
        ax3.imshow(draw_path(image_path, iba_path))
        ax3.set_title('IBA* Path')
        ax3.axis('off')

    plt.tight_layout()
    plt.savefig(f"{os.path.splitext(image_path)[0]}_results.png")
    plt.close()

    return {
        'Image': os.path.basename(image_path),
        'A*_Time': astar_time,
        'A*_Memory': astar_memory,
        'A*_Nodes': astar_nodes,
        'A*_Path_Length': len(astar_path) if astar_path else 0,
        'IBA*_Time': iba_time,
        'IBA*_Memory': iba_memory,
        'IBA*_Nodes': iba_nodes,
        'IBA*_Path_Length': len(iba_path) if iba_path else 0
    }

def analyze_results(results):
    df = pd.DataFrame(results)
    
    print("\nDataFrame Columns:")
    print(df.columns)
    
    # Calculate average performance metrics
    astar_cols = [col for col in df.columns if col.startswith('A*_')]
    iba_cols = [col for col in df.columns if col.startswith('IBA*_')]
    
    avg_performance = df[astar_cols + iba_cols].mean()
    
    # Create a summary table
    summary_data = {
        'Metric': [col.split('_', 1)[1] for col in astar_cols],
        'A*': avg_performance[astar_cols].values,
        'IBA*': avg_performance[iba_cols].values
    }
    summary_table = pd.DataFrame(summary_data)
    summary_table['Improvement (%)'] = ((summary_table['A*'] - summary_table['IBA*']) / summary_table['A*'] * 100).round(2)
    
    print("\nSummary Table:")
    print(summary_table.to_string(index=False))
    
    # Save the summary table as a CSV file
    summary_table.to_csv('pathfinding_summary.csv', index=False)
    
    # Create visualizations
    plt.figure(figsize=(15, 10))
    
    for i, (astar_col, iba_col) in enumerate(zip(astar_cols, iba_cols), 1):
        plt.subplot(2, 2, i)
        sns.boxplot(data=df[[astar_col, iba_col]])
        plt.title(f'{astar_col.split("_", 1)[1]} Comparison')
        plt.ylabel(astar_col.split("_", 1)[1])
    
    plt.tight_layout()
    plt.savefig('pathfinding_comparison.png')
    plt.close()
    
    # Performance Improvement Visualization
    improvement = ((df[astar_cols] - df[iba_cols]) / df[astar_cols] * 100)
    improvement.columns = [col.split('_', 1)[1] for col in astar_cols]
    
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=improvement)
    plt.title('Performance Improvement of IBA* over A*')
    plt.ylabel('Improvement (%)')
    plt.savefig('performance_improvement.png')
    plt.close()

def main(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            result = process_image(image_path)
            results.append(result)
            
            print("\nA* Results:")
            print(f"Execution time: {result['A*_Time']:.4f} seconds")
            print(f"Memory usage: {result['A*_Memory']:.2f} MB")
            print(f"Nodes visited: {result['A*_Nodes']}")
            print(f"Path length: {result['A*_Path_Length']}")

            print("\nOptimized Incremental A* Results:")
            print(f"Execution time: {result['IBA*_Time']:.4f} seconds")
            print(f"Memory usage: {result['IBA*_Memory']:.2f} MB")
            print(f"Nodes visited: {result['IBA*_Nodes']}")
            print(f"Path length: {result['IBA*_Path_Length']}")

    analyze_results(results)

if __name__ == "__main__":
    folder_path = r"grid_maps"
    main(folder_path)